In [2]:
import numpy as np
from scipy.io.wavfile import read as wav_read

def compute_loudness(file_path):
    """
    Compute the loudness of an audio file using RMS.
    :param file_path: Path to the audio file
    :return: Loudness in dB
    """
    # Load the audio file
    sample_rate, audio = wav_read(file_path)
    
    # If stereo, convert to mono
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Normalize the audio to range [-1, 1]
    audio = audio / np.max(np.abs(audio))
    
    # Compute RMS (Root Mean Square)
    rms = np.sqrt(np.mean(audio**2))
    
    # Convert RMS to dB
    loudness_db = 20 * np.log10(rms)
    
    return loudness_db

# Example usage
file_path = "Data/sine_wave_44100Hz_16bit_1min.wav"
loudness = compute_loudness(file_path)
print(f"Loudness: {loudness:.2f} dB")

# Example usage for the music file
file_path = "Data/CTHS3_Acid_Bass_Loop_01_C_123.wav"
loudness = compute_loudness(file_path)
print(f"Loudness: {loudness:.2f} dB")


Loudness: -3.01 dB
Loudness: -15.13 dB


/var/folders/b0/m80gf4wx3pvdbp860pz1fqwc0000gn/T/ipykernel_51552/2290662283.py:11: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio = wav_read(file_path)


In [6]:
import numpy as np
from scipy.io.wavfile import read as wav_read
from scipy.signal import lfilter

def k_weighting_filter(audio, sample_rate):
    """
    Apply K-weighting filter as per ITU-R BS.1770.
    :param audio: Input audio signal
    :param sample_rate: Sampling rate of the audio
    :return: Filtered audio signal
    """
    # Pre-emphasis filter coefficients for K-weighting
    b = [1, -1]
    a = [1, -0.999]
    pre_emphasized = lfilter(b, a, audio)
    return pre_emphasized

def compute_audio_metrics(file_path):
    """
    Compute LUFS, peak amplitude, crest factor, and loudness of an audio file.
    :param file_path: Path to the audio file
    :return: LUFS (dB), peak amplitude, crest factor (dB), loudness (dB)
    """
    # Load the audio file
    sample_rate, audio = wav_read(file_path)
    
    # If stereo, convert to mono
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Normalize the audio to range [-1, 1]
    audio = audio / np.max(np.abs(audio))
    
    # Apply K-weighting filter
    weighted_audio = k_weighting_filter(audio, sample_rate)
    
    # Compute RMS of the weighted audio
    rms_weighted = np.sqrt(np.mean(weighted_audio**2))
    
    # Compute LUFS
    lufs = 20 * np.log10(rms_weighted)
    
    # Compute peak amplitude
    peak_amplitude = np.max(np.abs(audio))
    
    # Compute crest factor
    crest_factor = 20 * np.log10(peak_amplitude / rms_weighted)
    
    # Compute RMS-based loudness
    rms = np.sqrt(np.mean(audio**2))
    loudness = 20 * np.log10(rms)
    
    return lufs, peak_amplitude, crest_factor, loudness

# Example usage for the sine wave file
file_path_sine = "Data/sine_wave_44100Hz_16bit_1min.wav"
lufs_sine, peak_amplitude_sine, crest_factor_sine, loudness_sine = compute_audio_metrics(file_path_sine)
print(f"Sine Wave - LUFS: {lufs_sine:.2f} dB, Peak Amplitude: {peak_amplitude_sine:.2f}, Crest Factor: {crest_factor_sine:.2f} dB, Loudness: {loudness_sine:.2f} dB")

# Example usage for the music file
file_path_music = "Data/CTHS3_Acid_Bass_Loop_01_C_123.wav"
lufs_music, peak_amplitude_music, crest_factor_music, loudness_music = compute_audio_metrics(file_path_music)
print(f"Music File - LUFS: {lufs_music:.2f} dB, Peak Amplitude: {peak_amplitude_music:.2f}, Crest Factor: {crest_factor_music:.2f} dB, Loudness: {loudness_music:.2f} dB")

Sine Wave - LUFS: -3.01 dB, Peak Amplitude: 1.00, Crest Factor: 3.01 dB, Loudness: -3.01 dB
Music File - LUFS: -15.17 dB, Peak Amplitude: 1.00, Crest Factor: 15.17 dB, Loudness: -15.13 dB


/var/folders/b0/m80gf4wx3pvdbp860pz1fqwc0000gn/T/ipykernel_51552/2277169083.py:25: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio = wav_read(file_path)
